Notebook pour calculer les Excess Waiting Time.
Tout d'abord on a besoin des packages pour Cluster

In [1]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# cluster visualizer
#%matplotlib inline
#from yellowbrick.cluster import KElbowVisualizer 

# sklearn kmeans
from sklearn.metrics.cluster import contingency_matrix

# pyclustering kmeans
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.kmedians import kmedians
from pyclustering.utils.metric import distance_metric
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.encoder import type_encoding
from pyclustering.cluster.encoder import cluster_encoder
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from numpy import unique
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture

import csv

In [2]:
#We import the schedule

path_gtfs ='C:/Users/Maison/Documents/INFO-H423/Projet Data Mining/GTFS3sep/gtfs3sept.zip'

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(path_gtfs,busiest_date=False)

In [3]:
#We import the calendar by hand 

calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')  #also export the exceptions

In [4]:
def FromDayToService(date):
    '''Determine all the service_id concerned by a date, given in the format date=('20210907','tuesday')
        it first take a look at every regular service_id then examine the exceptions. 
        Returns a list of service_id of type string'''

    ListServiceId=[]
    for i in range(len(calendardf)):
        startdate,endate=calendardf.iloc[i,[8,9]]
        if int(date[0]) in range(int(startdate),int(endate)+1):
            if calendardf.loc[i,date[1]]==1:
                #It appends every service_id which contains the date, and for which there is a '1'
                # under right day in the week
                ListServiceId.append(str(calendardf.iloc[i,0]))     

    #Now we have to take a look a the exceptions
    #We selections all the exceptions in calendarDatesdf that concerns the date in the input
    ModifServiceId= calendarDatesdf[calendarDatesdf['date']==int(date[0])][['service_id','exception_type']]
    #print('Number of service_id involved in modifications: '+ str(len(ModifServiceId)))
    #for every modif we check the kind of exception
    for modif in ModifServiceId.iterrows():  
        service = str(modif[1]['service_id'])
        exception = modif[1]['exception_type']
        if exception == 1:
            #if the exception is 1, it means that we should change to a 1 the cell related to the date in
            # input, in the row equal to service in calendar
            ListServiceId.append(service)
            #print('ServiceId: '+str(service)+' correclty added' )
        elif (exception == 2) & (service in ListServiceId):
            #if the exception is 2, it means that we should change to a 0 the cell related to the date in
            # input, in the row equal to service in calendar
            #Since this service_id must have been selected in the first loop (in Calendar), we need to remove it
            ListServiceId.remove(service)
            #print('ServiceId: '+str(service)+' correclty removed' )
        elif (exception == 2) & (service not in ListServiceId):
            #if the exception is 2 but we haven't found the service id in the first loop, this is weird
            #In principle this should not happend, but we're never too carefull
            print('Warning : exception 2 but no service_id = ' +service+ ' found in calendar')
    return (ListServiceId)

def FromLineIdtoRouteId(LineId):
    
    '''Takes a lineid in argument a returns the correspondent routeid, since stop_times only 
        contains route_id'''

    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def ScheduledTime(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. The schedule is composed as follows: if we are in a period of
        punctuality then returns the (arrivaltime,true), and if we are in regularity, 
        returns (TimeOfWaitFromPreviousVehicle,False), all in one list '''

    ListOfService = pd.Series(FromDayToService(date))  #List of service_id tha apply for that day
    #RouteId = FromLineIdtoRouteId(lineId)               #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Scheduledf['arrival_time'].values
    #print('Number of service_id involved: '+ str(len(ListOfService)))
    #We compute every wainting time to determine punctuality of regularity (< or > 720 s)
    #Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    #Schedule.insert(0,ArrivalTime[0])
    #True if punctuality, False if regularity
    Punctuality = np.array(IntervalTime)>720 
    #Ultimately, if we are a punctuality, we want the arrival time and if we are in regularity, we want the 
    #time bewteen two vehicles
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            if (0<i<(len(Punctuality)-1)):
                if ~(Punctuality[i-1]) & ~(Punctuality[i+1]):
                    Punctuality[i]=False
            else:
                IntervalTime[i]=ArrivalTime[i]
 
    #Schedule = list(zip(Schedule,Punctuality))
    #The format is a list of tuples
    if len(IntervalTime)!=len(ArrivalTime[:-1]):
        print('Attention: la longueur de Schedule et ArrivalTime ne match pas dans ScheduledTime ')
    return IntervalTime,Punctuality,ArrivalTime[:-1]

def RouteIdToLineId(routeId):

    '''Takes a routeid in argument a returns the correspondent lineid'''

    lineId = routes[routes['route_id']==routeId]['route_short_name'].iloc[0]
    return lineId

def TroncatedScheduleRegularity(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) in the format date=('20210907','tuesday') and returns the IntervalTime 
        and ArrivalTime cleared from the ponctuality point'''
    ScheduleNonCleared = ScheduledTime(date,routeid,stopId)
    TroncatedInterval = [intervaltime for i,intervaltime in enumerate(ScheduleNonCleared[0]) if not(ScheduleNonCleared[1][i])]
    TroncatedArrival = [arrivaltime for i,arrivaltime in enumerate(ScheduleNonCleared[2]) if not(ScheduleNonCleared[1][i])]
    if len(TroncatedInterval)!= len(TroncatedArrival):
        print('Attention: la longueur de TroncatedInterval et TroncatedArrival ne match pas dans TroncatedScheduleRegularity ')
    return TroncatedInterval,TroncatedArrival

Bellow is the cell that compute the middle stop from a line in a specific direction, not really relevant now

In [5]:
StopTimesGroupTrip = stop_times.groupby('trip_id')[['stop_sequence']].mean().round()
StopTimesGroupTrip['stop_sequence']=StopTimesGroupTrip['stop_sequence'].astype(int)

StopTimesOnlyMeanSeq = pd.merge(stop_times, StopTimesGroupTrip, how='right', on= ['trip_id','stop_sequence'])
StopTimesOnlyMeanSeq = StopTimesOnlyMeanSeq[['route_id','stop_sequence','stop_id','direction_id']]
DicOfMiddleStop={}

for route in StopTimesOnlyMeanSeq['route_id'].unique():
    StopTimeMeanSeqRoute = StopTimesOnlyMeanSeq[StopTimesOnlyMeanSeq['route_id']==route]
    directionIds = StopTimeMeanSeqRoute['direction_id'].unique()
    for direction in directionIds:
        #Ici je prends la première valeure que je trouve qui a la bonne route et direction
        # car il peut y avoir plusieurs candidat Middle stop pour chaque (ligne,direction)
        # ici je prend le premier parce que j'en ai très exactement rien à fouttre
        stop = StopTimeMeanSeqRoute[StopTimeMeanSeqRoute['direction_id']==direction]['stop_id'].values[0]
        value = str(route)+':'+str(direction)
        if DicOfMiddleStop.get(stop) is None:
            DicOfMiddleStop[stop]=value


#Format du dico : 'middle_stop_id': 'routeid:direction'    Ex: '5510': '41:0'

#Ici c'est juste pour lui dire quelle date prendre en exemple si on dit 'weekday', 'satuday', etc...
DicOfWeek = {'weekday':('20210915','wednesday'),'saturday':('20210918','saturday'),'sunday':('20210919','sunday')}

#liste des middle stop
ListOfMiddleStop= list(DicOfMiddleStop.keys())

def get_key(val):
    for key, value in DicOfMiddleStop.items():
        if val == value:
            return key


Now we need the algorithm to cluster 

In [6]:
# define dictionary for distance measures
distance_measures = {'euclidean': 0, 'squared euclidean': 1, 'manhattan': 2, 'chebyshev': 3, 
                    'canberra': 5, 'chi-square': 6}

# function defined to compute purity score using pyclustering for various distance measures
def Clustering(dist_measure,nclust,data):
    initial_centers = random_center_initializer(data.values, nclust, random_state=5).initialize()
    # instance created for respective distance metric
    instanceKm = kmeans(data.values, initial_centers=initial_centers, metric=distance_metric(dist_measure))
    # perform cluster analysis
    instanceKm.process()
    # cluster analysis results - clusters and centers
    pyClusters = instanceKm.get_clusters()
    pyCenters = instanceKm.get_centers()
    # enumerate encoding type to index labeling to get labels
    pyEncoding = instanceKm.get_cluster_encoding()
    pyEncoder = cluster_encoder(pyEncoding, pyClusters, data.values)
    pyLabels = pyEncoder.set_encoding(0).get_clusters()
    # function purity score is defined in previous section

    return(pyClusters,pyCenters)
    #return purity_score(IntervalTimeBetweenVehicle['TimeInterval'].values, pyLabels)

# print results
#for measure, value in distance_measures.items():
    #print(f"The purity score for {measure} distance is {round(pyPurity(value)*100, 2)}%")

D'abord on va venir mettre toute la data qui concerne cette date dans un CSV, par arrêt 

In [7]:
ListOfStopId  = list(stop_times['stop_id'].unique())
DicOfStopInRoute = {}
for route in routes['route_id']:
    ListOfStop = [stop for stop in stop_times[stop_times['route_id']==route]['stop_id'].unique()]
    DicOfStopInRoute[route]=ListOfStop

In [8]:
def ScheduledTimeCSV(date,routeid,stopId):

    '''Takes a triplet (date,lineId,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. THE DIFFERENCE WITH ScheduledTime IS THAT WE DONT
        NEED THE PUNCTUALITY'''

    ListOfService = pd.Series(FromDayToService(date))              #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    directionId = Scheduledf['direction_id'].unique()
    ArrivalTime = Scheduledf['arrival_time'].values
    

    return ArrivalTime,directionId

Schedule cell pour mecredi 

In [17]:
#Runtime ~ 30min

#Merecredi 15/09

headerSchedule = ['route_id','stop_id','ArrivalTime']

with open('Schedule20210915.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSchedule)
    k=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            ArrivalTime,directionId = ScheduledTimeCSV(('20210915','wednesday'),route,stop)
            for arrival in ArrivalTime:
                row = [route,stop,arrival]
                writer.writerow(row)
            
        k+=1
        if k%5==0:
            print(str(k)+' lignes encodées')
            k=0

Schedule Cell pour Vendredi

In [ ]:
#Runtime ~ 30min

#Vendredi 17/09

headerSchedule = ['route_id','stop_id','ArrivalTime']

with open('Schedule20210917.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSchedule)
    k=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            ArrivalTime,directionId = ScheduledTimeCSV(('20210917','friday'),route,stop)
            for arrival in ArrivalTime:
                row = [route,stop,arrival]
                writer.writerow(row)
            
        k+=1
        if k%5==0:
            print(str(k)+' lignes encodées')
            k=0

Schedule Cell pour Samedi

In [ ]:
#Runtime ~ 30min

#Samedi 18/09

headerSchedule = ['route_id','stop_id','ArrivalTime']

with open('Schedule20210918.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSchedule)
    k=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            ArrivalTime,directionId = ScheduledTimeCSV(('20210918','saturday'),route,stop)
            for arrival in ArrivalTime:
                row = [route,stop,arrival]
                writer.writerow(row)
            
        k+=1
        if k%5==0:
            print(str(k)+' lignes encodées')
            k=0

Schedule Cell pour Dimanche

In [ ]:
#Runtime ~ 30min

#Dimanche 19/09

headerSchedule = ['route_id','stop_id','ArrivalTime']

with open('Schedule20210919.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSchedule)
    k=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            ArrivalTime,directionId = ScheduledTimeCSV(('20210919','sunday'),route,stop)
            for arrival in ArrivalTime:
                row = [route,stop,arrival]
                writer.writerow(row)
            
        k+=1
        if k%5==0:
            print(str(k)+' lignes encodées')
            k=0

Pour l'exemple regardons au format d'une ligne du CSV

In [38]:
ReadSchedule = csv.reader(open('Schedule20210915.csv', 'r'),delimiter=',')

headerSchedule = ['route_id','direction_id','stop_id','ArrivalTime']

with open('SchedulePunc20210915.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(headerSchedule)
    for i,row in enumerate(ReadSchedule):
        
        if len(row)>4:
            ArrivalTime = (np.array(row[3:])).astype(np.float64)
            IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
            Punctuality = IntervalTime>720
            ArrivalTimePunct = [arrival for k,arrival in enumerate(ArrivalTime[:-1]) if Punctuality[k]]
            for arrivalnew in ArrivalTimePunct:
                RowToWrite=[row[0],row[2],arrivalnew]
                writer.writerow(RowToWrite)
        if i%200==0:
            print(str(i/2)+' arrêts encodés')




0.0 arrêts encodés
100.0 arrêts encodés
200.0 arrêts encodés
300.0 arrêts encodés
400.0 arrêts encodés
500.0 arrêts encodés
600.0 arrêts encodés
700.0 arrêts encodés
800.0 arrêts encodés
900.0 arrêts encodés
1000.0 arrêts encodés
1100.0 arrêts encodés
1200.0 arrêts encodés
1300.0 arrêts encodés
1400.0 arrêts encodés
1500.0 arrêts encodés
1600.0 arrêts encodés
1700.0 arrêts encodés
1800.0 arrêts encodés
1900.0 arrêts encodés
2000.0 arrêts encodés
2100.0 arrêts encodés
2200.0 arrêts encodés
2300.0 arrêts encodés
2400.0 arrêts encodés
2500.0 arrêts encodés
2600.0 arrêts encodés
2700.0 arrêts encodés
2800.0 arrêts encodés
2900.0 arrêts encodés
3000.0 arrêts encodés
3100.0 arrêts encodés
3200.0 arrêts encodés
3300.0 arrêts encodés
3400.0 arrêts encodés
3500.0 arrêts encodés
3600.0 arrêts encodés
3700.0 arrêts encodés
3800.0 arrêts encodés
3900.0 arrêts encodés
4000.0 arrêts encodés
4100.0 arrêts encodés
4200.0 arrêts encodés
4300.0 arrêts encodés
4400.0 arrêts encodés
4500.0 arrêts encodés


On peut mettre mtn l'ago de clustering

In [ ]:
routeidTram = ['3','10','6','11','12','25','76','41','40','15','26','9','42','14','16','7','75','8']
routeidMetro = ['2','4','1','3']
routeidBus = ['5', '13', '17', '18', '19', '20', '21', '22', '23', '24', '27', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '43', '44', '45', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '77', '78', '79', '80', '81', '82', '84', '85', '86', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97']


def ClusteringOfData(dist_measure,Data,DataDelta,routeid,dayoftheweek):

    #il faut sortir routeidMetro etc de la boucle
    Clusters = []
    routeidMetro = ['2','4','1','3']
    routeidTram = ['3','10','6','11','12','25','76','41','40','15','26','9','42','14','16','7','75','8']

    #delete if len(data)<10

    if routeid in routeidMetro:
        if dayoftheweek == 'weekday':
            nclust = 4
            nY = 1000
            coefCent = 1.05
        else:
            nclust = 3
            nY = 10000
            coefCent = 1.05
    elif routeid in routeidTram:
        if dayoftheweek == 'weekday':
            #print(len(Data))
            nclust = len(Data)//35
            nY = 1
            coefCent = 1
        else:
            #print(len(Data))
            nclust = len(Data)//25
            nY = 1000
            coefCent = 1
    else:
        if dayoftheweek == 'weekday':
            #print(len(Data))
            nclust = len(Data)//35
            nY = 1000
            coefCent = 1
        else:
            #print(len(Data))
            nclust = len(Data)//25
            nY = 1000
            coefCent = 1
    
    if nclust == 1 or nclust == 0:
        return(Clusters,nclust)

    maxX = np.max(DataDelta.iloc[:,:1])*1
    maxY = np.max(DataDelta.iloc[:,1:])*nY
    DataDelta.iloc[:,:1] = DataDelta.iloc[:,:1]/maxX
    DataDelta.iloc[:,1:] = DataDelta.iloc[:,1:]/maxY

    initial_centers = kmeans_plusplus_initializer(DataDelta.values, nclust,amount_candidates = 4).initialize()
    instanceKm = kmeans(DataDelta.values, initial_centers=initial_centers, metric=distance_metric(dist_measure))
    instanceKm.process()
    Centers = instanceKm.get_centers()

    for center in Centers:
        center[0] = center[0]*coefCent

    instanceKmedians = kmedians(DataDelta.values, initial_medians=Centers,tolerance = 0.001)
    instanceKmedians.process()
    Clusters = instanceKmedians.get_clusters() 


    return(Clusters,nclust)

In [ ]:
#Vieilles fonctions qui servent que à Gulli

def DataForCluster(routeid, directionid,dayoftheweek):
    key = get_key(str(routeid)+':'+str(directionid))
    if key!=None:
        middlestop = key
        dataCluster = TroncatedScheduleRegularity(DicOfWeek[dayoftheweek],routeid,middlestop)
        IntervalTime = dataCluster[0]
        ArrivalTime = dataCluster[1]
        df = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
    return df

def DataForClusterDelta(routeid, directionid,dayoftheweek):
    key = get_key(str(routeid)+':'+str(directionid))
    if key!=None:
        middlestop = key
        dataCluster = TroncatedScheduleRegularity(DicOfWeek[dayoftheweek],routeid,middlestop)
        ArrivalTime = dataCluster[1]
        df = pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
    return df

Ok maintenant on peut calculer les time groups, Mehdi s'en est chargé, ici un exemple du code pour Mercredi

In [ ]:
#Runtime ~ 35min 

#Cellule pour mecredi 15/09

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupWednesday20210915AllStop.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210915','wednesday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            DataDelta= pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
            if len(Data)>10:
                #ClusteringOfData(dist_measure,nclust,Data,DataDelta,routeid,dayoftheweek):
                ClustersOutput = ClusteringOfData(distance_measures['manhattan'],Data,DataDelta,route,'weekday')
                Clusters = ClustersOutput[0]
                NumCluster = ClustersOutput[1]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    #Warning we need to invert since every cluster is in other order
                    SWT = Data.iloc[cluster[-1]:cluster[0],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str(stop))
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[-1],0])+'::'+str(Data.iloc[cluster[0],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                for i in range(NumCluster,6):
                    row.append('0::0::0')
                writer.writerow(row)
        k+=1
        if k%10==0:
            print(str(k)+' routes encodées')

Regardons au format d'une ligne de TimeGroupWednesday20210915AllStop, que j'exporte en DataFrame

In [ ]:
Clusterdf = pd.read_csv('TimeGroupWednesday20210915AllStop.csv')
Clusterdf.head()

Format d'une cellule: on a une string avec 3 nombre séparés par des '::' . Les deux premiers sont les ranges de temps déterminé par le Cluster1, ici dans la première ligne c'est entre 19800 et 32447 secondes après minuit. Le dernier nombre est le SWT calculé dans ce cluster, via les intervales, voir slides STIB

Nickel ! Maintenant qu'on a ça il suffit de venir chercher les données JSON de Mehdi pour la même date, et de les trier par Cluster, c'est l'objet de la prochaine cellule

Classement JSON regularité Mercredi

In [ ]:
with open('TimeGroupWednesday20210915AllStop8Clust.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        

Problème y'a plus de cluster que ce qu'on pensait, ici 10, va falloir qu'on append des 0::0::0

In [18]:
ReaderTimeGroup = csv.reader(open('TimeGroupWednesday20210915AllStop.csv', 'r'),delimiter=',')

for i,row in enumerate(ReaderTimeGroup):
    if i==1:
        max = len(row)
    if i>1:
        if len(row)>max:
            max=len(row)
print(max)

12


In [23]:
ReaderTimeGroup = csv.reader(open('TimeGroupWednesday20210915AllStop.csv', 'r'),delimiter=',')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']
with open('TimeGroupWednesday20210915AllStopAdapatedClust.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    for i,row in enumerate(ReaderTimeGroup):
        if i>0:
            for k in range(len(row),12):
                row.append('0::0::0')
            writer.writerow(row)
            

In [24]:
#Runtime ~ 10sec  

ReaderJson = csv.reader(open('stop_times_line_wed.csv', 'r'),delimiter=',')
TimeGroupClusterdf = pd.read_csv('TimeGroupWednesday20210915AllStopAdapatedClust.csv')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']


#Fonction pour lire le CSV JSON

with open('ClassifiedRegularityPointInJsonWed.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        if len(row)>2:
            route = int(FromLineIdtoRouteId(str((row[0]))))
            stop = str(row[1])
            RangeClusterInvolved = TimeGroupClusterdf[(TimeGroupClusterdf['route_id']==route) & (TimeGroupClusterdf['stop_id']==stop) ][['Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']].values
            
            if len(RangeClusterInvolved)>0:
                RangeClusterInvolved = [RangeClust.split('::') for RangeClust in RangeClusterInvolved[0]]
                ArrivalTime = np.array(row[2].split(',')).astype(np.float)
                IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
                RowToWrite=[route,stop]
                for rangecluster in RangeClusterInvolved:
                    borneinf = float(rangecluster[0])
                    bornesup = float(rangecluster[1])
                    swt = rangecluster[2]
                    DataInCluster= str(rangecluster[2])  #We start writing the row with SWT factor that we'll need later
                    for i,interval in enumerate(IntervalTime):
                        if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup):
                            DataInCluster+= ','+str(interval)
                    #DataInCluster = [IntervalTime[i] for i in range(len(IntervalTime))] #if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup) ]
                    RowToWrite.append(DataInCluster)
                writer.writerow(RowToWrite)
            
        k+=1
        if k==500:
            print(str(k*l)+' arrêt encodées')
            l+=1
            k=0

500 arrêt encodées
1000 arrêt encodées
1500 arrêt encodées
2000 arrêt encodées


Classement JSON regularité Vendredi

In [25]:
ReaderTimeGroup = csv.reader(open('TimeGroupFriday20210917AllStop.csv', 'r'),delimiter=',')

for i,row in enumerate(ReaderTimeGroup):
    if i==1:
        maxFri = len(row)
    if i>1:
        if len(row)>maxFri:
            maxFri=len(row)
print(maxFri)

12


In [26]:
ReaderTimeGroup = csv.reader(open('TimeGroupFriday20210917AllStop.csv', 'r'),delimiter=',')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']
with open('TimeGroupFriday20210917AllStopAdapatedClust.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    for i,row in enumerate(ReaderTimeGroup):
        if i>0:
            for k in range(len(row),12):
                row.append('0::0::0')
            writer.writerow(row)

In [28]:
#Runtime ~ 10sec  

ReaderJson = csv.reader(open('stop_times_line_fri.csv', 'r'),delimiter=',')
TimeGroupClusterdf = pd.read_csv('TimeGroupFriday20210917AllStopAdapatedClust.csv')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']

#Fonction pour lire le CSV JSON

with open('ClassifiedRegularityPointInJsonFri.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        if len(row)>2:
            route = int(FromLineIdtoRouteId(str((row[0]))))
            stop = str(row[1])
            RangeClusterInvolved = TimeGroupClusterdf[(TimeGroupClusterdf['route_id']==route) & (TimeGroupClusterdf['stop_id']==stop) ][['Cluster1','Cluster2','Cluster3','Cluster4','Cluster5','Cluster6']].values
            
            if len(RangeClusterInvolved)>0:
                RangeClusterInvolved = [RangeClust.split('::') for RangeClust in RangeClusterInvolved[0]]
                ArrivalTime = np.array(row[2].split(',')).astype(np.float)
                IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
                RowToWrite=[route,stop]
                for rangecluster in RangeClusterInvolved:
                    borneinf = float(rangecluster[0])
                    bornesup = float(rangecluster[1])
                    swt = rangecluster[2]
                    DataInCluster= str(rangecluster[2])  #We start writing the row with SWT factor that we'll need later
                    for i,interval in enumerate(IntervalTime):
                        if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup):
                            DataInCluster+= ','+str(interval)
                    #DataInCluster = [IntervalTime[i] for i in range(len(IntervalTime))] #if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup) ]
                    RowToWrite.append(DataInCluster)
                writer.writerow(RowToWrite)
            
        k+=1
        if k==500:
            print(str(k*l)+' arrêt encodées')
            l+=1
            k=0

500 arrêt encodées
1000 arrêt encodées
1500 arrêt encodées
2000 arrêt encodées
2500 arrêt encodées


Classement JSON regularité Samedi

In [33]:
ReaderTimeGroup = csv.reader(open('TimeGroupSaturday20210918AllStop.csv', 'r'),delimiter=',')

for i,row in enumerate(ReaderTimeGroup):
    if i==1:
        maxSat = len(row)
    if i>1:
        if len(row)>maxSat:
            maxSat=len(row)
print(maxSat)

14


In [34]:
ReaderTimeGroup = csv.reader(open('TimeGroupSaturday20210918AllStop.csv', 'r'),delimiter=',')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10','Cluster11','Cluster12']
with open('TimeGroupSaturday20210917AllStopAdapatedClust.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    for i,row in enumerate(ReaderTimeGroup):
        if i>0:
            for k in range(len(row),14):
                row.append('0::0::0')
            writer.writerow(row)

In [35]:
#Runtime ~ 10sec  

ReaderJson = csv.reader(open('stop_times_line_sat.csv', 'r'),delimiter=',')
TimeGroupClusterdf = pd.read_csv('TimeGroupSaturday20210917AllStopAdapatedClust.csv')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10']


#Fonction pour lire le CSV JSON

with open('ClassifiedRegularityPointInJsonSat.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        if len(row)>2:
            route = int(FromLineIdtoRouteId(str((row[0]))))
            stop = str(row[1])
            RangeClusterInvolved = TimeGroupClusterdf[(TimeGroupClusterdf['route_id']==route) & (TimeGroupClusterdf['stop_id']==stop) ][['Cluster1','Cluster2','Cluster3','Cluster4','Cluster5','Cluster6']].values
            
            if len(RangeClusterInvolved)>0:
                RangeClusterInvolved = [RangeClust.split('::') for RangeClust in RangeClusterInvolved[0]]
                ArrivalTime = np.array(row[2].split(',')).astype(np.float)
                IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
                RowToWrite=[route,stop]
                for rangecluster in RangeClusterInvolved:
                    borneinf = float(rangecluster[0])
                    bornesup = float(rangecluster[1])
                    swt = rangecluster[2]
                    DataInCluster= str(rangecluster[2])  #We start writing the row with SWT factor that we'll need later
                    for i,interval in enumerate(IntervalTime):
                        if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup):
                            DataInCluster+= ','+str(interval)
                    #DataInCluster = [IntervalTime[i] for i in range(len(IntervalTime))] #if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup) ]
                    RowToWrite.append(DataInCluster)
                writer.writerow(RowToWrite)
            
        k+=1
        if k==500:
            print(str(k*l)+' arrêt encodées')
            l+=1
            k=0

500 arrêt encodées
1000 arrêt encodées
1500 arrêt encodées
2000 arrêt encodées
2500 arrêt encodées


Classement JSON regularité Dimanche

In [36]:
ReaderTimeGroup = csv.reader(open('TimeGroupSunday20210919AllStop.csv', 'r'),delimiter=',')

for i,row in enumerate(ReaderTimeGroup):
    if i==1:
        maxSun = len(row)
    if i>1:
        if len(row)>maxSun:
            maxSun=len(row)
print(maxSun)

14


In [37]:
ReaderTimeGroup = csv.reader(open('TimeGroupSunday20210919AllStop.csv', 'r'),delimiter=',')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10','Cluster11','Cluster12']
with open('TimeGroupSunday20210917AllStopAdapatedClust.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    for i,row in enumerate(ReaderTimeGroup):
        if i>0:
            for k in range(len(row),14):
                row.append('0::0::0')
            writer.writerow(row)

In [38]:
#Runtime ~ 10sec  

ReaderJson = csv.reader(open('stop_times_line_sun.csv', 'r'),delimiter=',')
TimeGroupClusterdf = pd.read_csv('TimeGroupSunday20210917AllStopAdapatedClust.csv')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6','Cluster7','Cluster8','Cluster9','Cluster10','Cluster11','Cluster12']

#Fonction pour lire le CSV JSON

with open('ClassifiedRegularityPointInJsonSun.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        if len(row)>2:
            route = int(FromLineIdtoRouteId(str((row[0]))))
            stop = str(row[1])
            RangeClusterInvolved = TimeGroupClusterdf[(TimeGroupClusterdf['route_id']==route) & (TimeGroupClusterdf['stop_id']==stop) ][['Cluster1','Cluster2','Cluster3','Cluster4','Cluster5','Cluster6']].values
            
            if len(RangeClusterInvolved)>0:
                RangeClusterInvolved = [RangeClust.split('::') for RangeClust in RangeClusterInvolved[0]]
                ArrivalTime = np.array(row[2].split(',')).astype(np.float)
                IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
                RowToWrite=[route,stop]
                for rangecluster in RangeClusterInvolved:
                    borneinf = float(rangecluster[0])
                    bornesup = float(rangecluster[1])
                    swt = rangecluster[2]
                    DataInCluster= str(rangecluster[2])  #We start writing the row with SWT factor that we'll need later
                    for i,interval in enumerate(IntervalTime):
                        if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup):
                            DataInCluster+= ','+str(interval)
                    #DataInCluster = [IntervalTime[i] for i in range(len(IntervalTime))] #if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup) ]
                    RowToWrite.append(DataInCluster)
                writer.writerow(RowToWrite)
            
        k+=1
        if k==500:
            print(str(k*l)+' arrêt encodées')
            l+=1
            k=0

500 arrêt encodées
1000 arrêt encodées
1500 arrêt encodées
2000 arrêt encodées
2500 arrêt encodées


Pour chaque ligne, on a une liste qui contient la route_id, le stop_id, puis 6 différentes string qui correspondent au 6 clusters et dans chaque string on a les intervals time du JSON qui sont contenus dans ce cluster, et qui sont séparés par une ','
ATTENTION: le dernier éléments de chaque string de cluster correspond au SWT

On peut calculer les EWT mtn !

In [39]:
def excesswaitingtime(swt,ArrivalTimeJSONArray):
    #both are given in second 
    #we give the EWT in second
    #ArrivalTimeJSONArray = np.array(ArrivalTimeJSONList)
    if swt == 'Problem with SWT':
        print('flag')
        return 'No EWT available'
    elif np.float(swt) ==np.float(0):
        return 'Problem with SWT = 0'
    elif sum(ArrivalTimeJSONArray)==0:
          return 'Problem with AWT = 0'
    else:
        awt = sum((ArrivalTimeJSONArray)**2)/(2*sum(ArrivalTimeJSONArray))
        ewt = awt-np.float(swt)
        return ewt
    

Celulle pour mecredi

In [40]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJsonWed.csv', 'r'),delimiter=',')

header = ['route_id','stop_id','EWT1', 'EWT2', 'EWT3', 'EWT4','EWT5','EWT6','EWT7','EWT8','EWT9','EWT10']#,'EWT11','EWT12']


with open('EWTScoreClusterAllStopWed20210915.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    # write the data
    
    for i,row in enumerate(ReaderClassifiedJson):
        route = row[0]
        stop = row[1]
        if i>0:
            RowToWrite=[route,stop]
            for IntervalInClusterReal in row[2:]:
                ListOfData = np.array(IntervalInClusterReal.split(','))
                swt = ListOfData[0]
                if (ListOfData[0] == 'Problem with SWT' ):
                    ewt = 'No EWT available'
                else:
                    ListOfData = ListOfData.astype(np.float)
                    ewt = excesswaitingtime(swt,ListOfData[1:])
                RowToWrite.append(ewt)
            writer.writerow(RowToWrite)

        if i%500==0:
            print(str(i)+' arrêts encodés')   

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés


Celulle pour vendredi 

In [41]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJsonFri.csv', 'r'),delimiter=',')

header = ['route_id','stop_id','EWT1', 'EWT2', 'EWT3', 'EWT4','EWT5','EWT6','EWT7','EWT8','EWT9','EWT10']#,'EWT11','EWT12']


with open('EWTScoreClusterAllStopFri20210917.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    # write the data
    
    for i,row in enumerate(ReaderClassifiedJson):
        route = row[0]
        stop = row[1]
        if i>0:
            RowToWrite=[route,stop]
            for IntervalInClusterReal in row[2:]:
                ListOfData = np.array(IntervalInClusterReal.split(','))
                swt = ListOfData[0]
                if (ListOfData[0] == 'Problem with SWT' ):
                    ewt = 'No EWT available'
                else:
                    ListOfData = ListOfData.astype(np.float)
                    ewt = excesswaitingtime(swt,ListOfData[1:])
                RowToWrite.append(ewt)
            writer.writerow(RowToWrite)

        if i%500==0:
            print(str(i)+' arrêts encodés')   

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés
2000 arrêts encodés


Cellule pour samedi

In [42]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJsonSat.csv', 'r'),delimiter=',')

header = ['route_id','stop_id','EWT1', 'EWT2', 'EWT3', 'EWT4','EWT5','EWT6','EWT7','EWT8','EWT9','EWT10','EWT11','EWT12']


with open('EWTScoreClusterAllStopSat20210918.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    # write the data
    
    for i,row in enumerate(ReaderClassifiedJson):
        route = row[0]
        stop = row[1]
        if i>0:
            RowToWrite=[route,stop]
            for IntervalInClusterReal in row[2:]:
                ListOfData = np.array(IntervalInClusterReal.split(','))
                swt = ListOfData[0]
                if (ListOfData[0] == 'Problem with SWT' ):
                    ewt = 'No EWT available'
                else:
                    ListOfData = ListOfData.astype(np.float)
                    ewt = excesswaitingtime(swt,ListOfData[1:])
                RowToWrite.append(ewt)
            writer.writerow(RowToWrite)

        if i%500==0:
            print(str(i)+' arrêts encodés')   

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés


Cellule pour dimanche

In [43]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJsonSun.csv', 'r'),delimiter=',')

header = ['route_id','stop_id','EWT1', 'EWT2', 'EWT3', 'EWT4','EWT5','EWT6','EWT7','EWT8','EWT9','EWT10','EWT11','EWT12']


with open('EWTScoreClusterAllStopSun20210919.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    # write the data
    
    for i,row in enumerate(ReaderClassifiedJson):
        route = row[0]
        stop = row[1]
        if i>0:
            RowToWrite=[route,stop]
            for IntervalInClusterReal in row[2:]:
                ListOfData = np.array(IntervalInClusterReal.split(','))
                swt = ListOfData[0]
                if (ListOfData[0] == 'Problem with SWT' ):
                    ewt = 'No EWT available'
                else:
                    ListOfData = ListOfData.astype(np.float)
                    ewt = excesswaitingtime(swt,ListOfData[1:])
                RowToWrite.append(ewt)
            writer.writerow(RowToWrite)

        if i%500==0:
            print(str(i)+' arrêts encodés')   

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés


In [44]:
EWTWednesday = pd.read_csv('EWTScoreClusterAllStopWed20210915.csv')
EWTWednesday[EWTWednesday['route_id']==40]

,route_id,stop_id,EWT1,EWT2,EWT3,EWT4,EWT5,EWT6,EWT7,EWT8,EWT9,EWT10
630,40,5501,252.31232507131497,204.93871957358945,179.6591890109118,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
631,40,5502,162.6432619263684,254.6497106185377,43.39070072533252,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
632,40,5503,139.17948646998937,308.60394411086156,166.07201091536552,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
633,40,5504,59.20488408864807,159.5200858115408,201.0150325554062,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
634,40,5507,514.8597141779827,385.7428391990361,334.9936392746641,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
635,40,5508,266.7793803305202,256.5010629722651,61.8116041974848,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
636,40,5601,420.7779671842627,329.1189543157203,389.20790657031785,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
637,40,5602,453.390120970091,860.9292724772914,290.0640872846379,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
638,40,5603,352.8961787372409,185.27744243659063,199.6674637264499,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0
639,40,5604,274.7872503114272,233.6677217913498,178.76900264542368,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0,Problem with SWT = 0


Ici on va juste modifier légèrement les outputs de stop line pour pouvoir les mettres en data frame, c'est utile pour l'analyse de Victoria

Cellule pour mercredi 

In [9]:
ReadJSON = csv.reader(open('stop_times_line_wed.csv','r'),delimiter=',')
header = ['route_id','stop_id','ArrivalTime']

with open('stop_time_line_wed_forDF.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    for i,rowJSON in enumerate(ReadJSON):
        if len(rowJSON)>2:
            route = FromLineIdtoRouteId(rowJSON[0])
            stop = rowJSON[1]
            ArrivalTime = np.array(rowJSON[2].split(',')).astype(np.float64)
            #IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
            for arrivalJSON in ArrivalTime:
                RowToWrite=[route,stop,arrivalJSON]
                writer.writerow(RowToWrite)
        if i%100==0:
            print(str(i)+' arrêts encodés')

0 arrêts encodés
100 arrêts encodés
200 arrêts encodés
300 arrêts encodés
400 arrêts encodés
500 arrêts encodés
600 arrêts encodés
700 arrêts encodés
800 arrêts encodés
900 arrêts encodés
1000 arrêts encodés
1100 arrêts encodés
1200 arrêts encodés
1300 arrêts encodés
1400 arrêts encodés
1500 arrêts encodés
1600 arrêts encodés
1700 arrêts encodés
1800 arrêts encodés
1900 arrêts encodés
2000 arrêts encodés
2100 arrêts encodés
2200 arrêts encodés


Cellule pour vendredi

In [10]:
ReadJSON = csv.reader(open('stop_times_line_fri.csv','r'),delimiter=',')
header = ['route_id','stop_id','ArrivalTime']

with open('stop_time_line_fri_forDF.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    for i,rowJSON in enumerate(ReadJSON):
        if len(rowJSON)>2:
            route = FromLineIdtoRouteId(rowJSON[0])
            stop = rowJSON[1]
            ArrivalTime = np.array(rowJSON[2].split(',')).astype(np.float64)
            #IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
            for arrivalJSON in ArrivalTime:
                RowToWrite=[route,stop,arrivalJSON]
                writer.writerow(RowToWrite)
        if i%500==0:
            print(str(i)+' arrêts encodés')

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés
2000 arrêts encodés
2500 arrêts encodés


Cellule pour samedi

In [11]:
ReadJSON = csv.reader(open('stop_times_line_sat.csv','r'),delimiter=',')
header = ['route_id','stop_id','ArrivalTime']

with open('stop_time_line_sat_forDF.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    for i,rowJSON in enumerate(ReadJSON):
        if len(rowJSON)>2:
            route = FromLineIdtoRouteId(rowJSON[0])
            stop = rowJSON[1]
            ArrivalTime = np.array(rowJSON[2].split(',')).astype(np.float64)
            #IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
            for arrivalJSON in ArrivalTime:
                RowToWrite=[route,stop,arrivalJSON]
                writer.writerow(RowToWrite)
        if i%500==0:
            print(str(i)+' arrêts encodés')

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés
2000 arrêts encodés
2500 arrêts encodés


Cellule pour dimanche

In [12]:
ReadJSON = csv.reader(open('stop_times_line_sun.csv','r'),delimiter=',')
header = ['route_id','stop_id','ArrivalTime']

with open('stop_time_line_sun_forDF.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    for i,rowJSON in enumerate(ReadJSON):
        if len(rowJSON)>2:
            route = FromLineIdtoRouteId(rowJSON[0])
            stop = rowJSON[1]
            ArrivalTime = np.array(rowJSON[2].split(',')).astype(np.float64)
            #IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
            for arrivalJSON in ArrivalTime:
                RowToWrite=[route,stop,arrivalJSON]
                writer.writerow(RowToWrite)
        if i%500==0:
            print(str(i)+' arrêts encodés')

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés
2000 arrêts encodés
2500 arrêts encodés


In [84]:
SchedulePuncdf = pd.read_csv('SchedulePunc20210915.csv')
JSONArrival = pd.read_csv('stop_time_line_cleared2_forDF.csv')
JSONArrival[(JSONArrival['stop_id']=='2')&(JSONArrival['route_id']==8161)]
JSONArrival['stop_id'].unique()

array([8011, 8012, 8021, ..., 5964, 9649, 9686], dtype=int64)

On peut mtn créer les fichier JSON de ponctualités avec les différences !

Cellule pour mecredi

In [14]:
#Fonction Vic ponctualité

SchedulePuncdf = pd.read_csv('Schedule20210915.csv')
JSONArrival = pd.read_csv('stop_time_line_wed_forDF.csv')

#ReadSchedule = csv.reader(open('SchedulePunc20210915.csv', 'r'),delimiter=',')
#ReadJSON = csv.reader(open('stop_times_line_cleared2.csv','r'),delimiter=',')


header = ['route_id','stop_id','ArrivalTime','Difference']

with open('PunctualityExploreWed.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    l=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            if stop.isnumeric():
                SchedulePuncdfForStop = SchedulePuncdf[(SchedulePuncdf['route_id']==int(route)) & (SchedulePuncdf['stop_id']==str(stop))]
                if len(SchedulePuncdfForStop)>0:
                    JSONArrivaldfForStop = JSONArrival[(JSONArrival['stop_id']==int(stop))&(JSONArrival['route_id']==int(route))]
                    if len(JSONArrivaldfForStop)>0:
                        ArrivalTime = JSONArrivaldfForStop['ArrivalTime'].values
                        for arrival in ArrivalTime:
                                closest = min(SchedulePuncdfForStop['ArrivalTime'].values,key=lambda x: abs(x-arrival))
                                difference = arrival-closest
                                RowToWrite=[route,stop,arrival,difference]
                                writer.writerow(RowToWrite)
                            
        l+=1
        if l%10==0:
            print(str(l)+' routes encodés')
                

10 arrêts encodés
20 arrêts encodés
30 arrêts encodés
40 arrêts encodés
50 arrêts encodés
60 arrêts encodés
70 arrêts encodés
80 arrêts encodés
90 arrêts encodés


In [23]:
MeasurePunc = pd.read_csv('PunctualityExplore.csv')
MeasurePunc[MeasurePunc['route_id']==40].head()

,route_id,stop_id,ArrivalTime,Difference
51177,40,5501,22400.964,18.964
51178,40,5501,23400.621,58.621
51179,40,5501,24589.724,-48.276
51180,40,5501,25129.592,47.592
51181,40,5501,25847.846,45.846


Cellule pour vendredi

In [20]:
#Fonction Vic ponctualité

SchedulePuncdf = pd.read_csv('Schedule20210917.csv')
JSONArrival = pd.read_csv('stop_time_line_fri_forDF.csv')

#ReadSchedule = csv.reader(open('SchedulePunc20210915.csv', 'r'),delimiter=',')
#ReadJSON = csv.reader(open('stop_times_line_cleared2.csv','r'),delimiter=',')


header = ['route_id','stop_id','ArrivalTime','Difference']

with open('PunctualityExploreFri.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    l=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            if stop.isnumeric():
                SchedulePuncdfForStop = SchedulePuncdf[(SchedulePuncdf['route_id']==int(route)) & (SchedulePuncdf['stop_id']==str(stop))]
                if len(SchedulePuncdfForStop)>0:
                    JSONArrivaldfForStop = JSONArrival[(JSONArrival['stop_id']==int(stop))&(JSONArrival['route_id']==int(route))]
                    if len(JSONArrivaldfForStop)>0:
                        ArrivalTime = JSONArrivaldfForStop['ArrivalTime'].values
                        for arrival in ArrivalTime:
                                closest = min(SchedulePuncdfForStop['ArrivalTime'].values,key=lambda x: abs(x-arrival))
                                difference = arrival-closest
                                RowToWrite=[route,stop,arrival,difference]
                                writer.writerow(RowToWrite)
                            
        l+=1
        if l%10==0:
            print(str(l)+' routes encodés')
                

10 routes encodés
20 routes encodés
30 routes encodés
40 routes encodés
50 routes encodés
60 routes encodés
70 routes encodés
80 routes encodés
90 routes encodés


Cellule pour samedi

In [19]:
#Fonction Vic ponctualité

SchedulePuncdf = pd.read_csv('Schedule20210918.csv')
JSONArrival = pd.read_csv('stop_time_line_sat_forDF.csv')

#ReadSchedule = csv.reader(open('SchedulePunc20210915.csv', 'r'),delimiter=',')
#ReadJSON = csv.reader(open('stop_times_line_cleared2.csv','r'),delimiter=',')


header = ['route_id','stop_id','ArrivalTime','Difference']

with open('PunctualityExploreSat.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    l=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            if stop.isnumeric():
                SchedulePuncdfForStop = SchedulePuncdf[(SchedulePuncdf['route_id']==int(route)) & (SchedulePuncdf['stop_id']==str(stop))]
                if len(SchedulePuncdfForStop)>0:
                    JSONArrivaldfForStop = JSONArrival[(JSONArrival['stop_id']==int(stop))&(JSONArrival['route_id']==int(route))]
                    if len(JSONArrivaldfForStop)>0:
                        ArrivalTime = JSONArrivaldfForStop['ArrivalTime'].values
                        for arrival in ArrivalTime:
                                closest = min(SchedulePuncdfForStop['ArrivalTime'].values,key=lambda x: abs(x-arrival))
                                difference = arrival-closest
                                RowToWrite=[route,stop,arrival,difference]
                                writer.writerow(RowToWrite)
                            
        l+=1
        if l%10==0:
            print(str(l)+' routes encodés')
                

10 routes encodés
20 routes encodés
30 routes encodés
40 routes encodés
50 routes encodés
60 routes encodés
70 routes encodés
80 routes encodés
90 routes encodés


Cellule pour dimanche

In [17]:
#Fonction Vic ponctualité

SchedulePuncdf = pd.read_csv('Schedule20210919.csv')
JSONArrival = pd.read_csv('stop_time_line_sun_forDF.csv')

#ReadSchedule = csv.reader(open('SchedulePunc20210915.csv', 'r'),delimiter=',')
#ReadJSON = csv.reader(open('stop_times_line_cleared2.csv','r'),delimiter=',')


header = ['route_id','stop_id','ArrivalTime','Difference']

with open('PunctualityExploreSun.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    l=0
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            if stop.isnumeric():
                SchedulePuncdfForStop = SchedulePuncdf[(SchedulePuncdf['route_id']==int(route)) & (SchedulePuncdf['stop_id']==str(stop))]
                if len(SchedulePuncdfForStop)>0:
                    JSONArrivaldfForStop = JSONArrival[(JSONArrival['stop_id']==int(stop))&(JSONArrival['route_id']==int(route))]
                    if len(JSONArrivaldfForStop)>0:
                        ArrivalTime = JSONArrivaldfForStop['ArrivalTime'].values
                        for arrival in ArrivalTime:
                                closest = min(SchedulePuncdfForStop['ArrivalTime'].values,key=lambda x: abs(x-arrival))
                                difference = arrival-closest
                                RowToWrite=[route,stop,arrival,difference]
                                writer.writerow(RowToWrite)
                            
        l+=1
        if l%10==0:
            print(str(l)+' routes encodés')
                

10 routes encodés
20 routes encodés
30 routes encodés
40 routes encodés
50 routes encodés
60 routes encodés
70 routes encodés
80 routes encodés
90 routes encodés


In [18]:
MeasurePunc = pd.read_csv('PunctualityExploreSun.csv')
MeasurePunc[MeasurePunc['route_id']==40].head()

,route_id,stop_id,ArrivalTime,Difference
55829,40,5501,21079.163,17.163
55830,40,5501,22865.202,3.202
55831,40,5501,27080.983,18.983
55832,40,5501,29493.056,31.056
55833,40,5501,30698.942,36.942
